# <font color='Red'>DDB (DESHIMA DATABASE)</font>

## Import Modules

In [1]:
import os
from pathlib import Path

import yaml
import numpy as np
from datetime import datetime
from astropy.io import fits
from astropy.io import ascii
from astropy import table
from astropy import coordinates
from astropy import units as u

In [2]:
D_ASTE    = (10.0* u.m).value                            # diameter of the ASTE
LON_ASTE  = coordinates.Angle('-67d42m11.89525s').deg    # longitude of the ASTE
LAT_ASTE  = coordinates.Angle('-22d58m17.69447s').deg    # latitude of the ASTE
PATH_DDBDICT = Path('/Users/tetsu/DESHIMA/devtools/merge/DB/DDB_dict.yaml').expanduser()
FORM_FITSTIME  = '%Y-%m-%dT%H:%M:%S'

In [3]:
def convert_fitstime(time_lis):
    if len(time_lis[0]) <= 14:
        return [datetime.strftime(datetime.strptime(time_lis[i], '%Y%m%d%H%M%S'),
                                  FORM_FITSTIME)
                for i in range(len(time_lis))]
    else:
        return [datetime.strftime(datetime.strptime(time_lis[i], '%Y%m%d%H%M%S.%f'),
                                  FORM_FITSTIME + '.%f')
                for i in range(len(time_lis))]
    
def addHeaderComments(hdr, key, com):
    hdr.comments[key] = com


def createBinTableHdu(data_dict):
    header = fits.Header()
    for (i, j, k) in zip(data_dict['hdr_key_lis'], data_dict['hdr_val_lis'], data_dict['hdr_com_lis']):
        header[i] = j, k

    columns = [fits.Column(name=data_dict['cols_key_lis'][i],
                           format=data_dict['tform'][i],
                           array=data_dict['cols_data_lis'][i],
                           unit=data_dict['tunit'][i])
               for i in range(len(data_dict['cols_key_lis']))]

    hdu = fits.BinTableHDU.from_columns(columns, header)
    [addHeaderComments(hdu.header, list(hdu.header)[i], data_dict['hdr_com_lis'][i])
     for i in range(- (len(data_dict['hdr_com_lis']) - data_dict['hdr_com_lis'].index('label for field 1')), 0)]

    return hdu

In [4]:
 def make_kidsinfo(kidsinfo_data):
    filters_data = Path(kidsinfo_data).expanduser()
    data = np.load(filters_data)

    pixelid   = np.zeros(113)
#    kidid     = data['kidid']
    kidid     = np.arange(0, 113)
#    kidtypes  = np.append(np.append(np.zeros(44), np.ones(4)), np.ones(1)* 2)
    kidtypes  = np.zeros(113)
#    bandpass  = data['bandpass']
    bandpass  = np.zeros((113, 400))
#    lorentz   = data['lorentz']
    lorentz   = np.zeros((113, 4))


    tform4 = str(len(bandpass[0])) + 'D'
    
    filename = os.path.basename(filters_data)
#    crval3   = 325.0e+09
    crval3   = 0.0
    crpix3   = 0.0
#    cdelt3   = 10.0e+06
    cdelt3   = 0.0

    with open(PATH_DDBDICT, 'r') as f:
        ddb_dict = yaml.load(f)

    kidsinfo_dict = ddb_dict['kidsinfo_dict']

    kidsinfo_dict['hdr_val_lis'][1] = filename
    kidsinfo_dict['hdr_val_lis'][6:] = [crval3, crpix3, cdelt3]
    kidsinfo_dict['cols_data_lis'] = [pixelid, kidid, kidtypes, bandpass, lorentz]
    kidsinfo_dict['tform'][3] = tform4
    
    return createBinTableHdu(kidsinfo_dict)

In [5]:
 def make_rcp():
#    filters_data = Path(rcp_data).expanduser()

    pixelid   = np.arange(0, 1)
    offsetaz  = np.zeros(1)
    offsetel  = np.zeros(1)
    gain      = np.ones(1)
    
    filename = None

    with open(PATH_DDBDICT, 'r') as f:
        ddb_dict = yaml.load(f)

    rcp_dict = ddb_dict['rcp_dict']

    rcp_dict['hdr_val_lis'][1] = filename
    rcp_dict['cols_data_lis'] = [pixelid, offsetaz, offsetel, gain]

    return createBinTableHdu(rcp_dict)

In [6]:
hdus = fits.HDUList()
hdus.append(fits.PrimaryHDU())
#hdus['PRIMARY'].header['DDB_ID'] = '20171018151938', 'The ID of DESHIMA DataBase'
hdus['PRIMARY'].header['DDB_ID'] = '20171022210533', 'The ID of DESHIMA DataBase'
hdus.append(make_kidsinfo('~/DESHIMA/datafile/Filterbank/filterbank_model.npz'))
hdus.append(make_rcp())

In [7]:
hdus.info()

Filename: (No file associated with this HDUList)
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU       5   ()      
  1  KIDSINFO    BinTableHDU     29   113R x 5C   ['K', 'K', 'K', '400D', '4D']   
  2  RCP         BinTableHDU     21   1R x 4C   ['K', 'D', 'D', 'D']   


In [8]:
table.Table(hdus['KIDSINFO'].data)

pixelid,kidid,kidtypes,bandpass [400],lorentz [4]
int64,int64,int64,float64,float64
0,0,0,0.0 .. 0.0,0.0 .. 0.0
0,1,0,0.0 .. 0.0,0.0 .. 0.0
0,2,0,0.0 .. 0.0,0.0 .. 0.0
0,3,0,0.0 .. 0.0,0.0 .. 0.0
0,4,0,0.0 .. 0.0,0.0 .. 0.0
0,5,0,0.0 .. 0.0,0.0 .. 0.0
0,6,0,0.0 .. 0.0,0.0 .. 0.0
0,7,0,0.0 .. 0.0,0.0 .. 0.0
0,8,0,0.0 .. 0.0,0.0 .. 0.0


In [9]:
ddb_fitsname = 'DDB_' + hdus['PRIMARY'].header['DDB_ID'] + '.fits'
#hdus.writeto('~/DESHIMA/datafile/' + ddb_fitsname)
hdus.writeto('/Users/tetsu/DESHIMA/devtools/merge/DB/' + ddb_fitsname)